## Part of kaggle competition Store Sales - Time Series Forecasting

This was my first kaggle competition ever and think i did a really good job in creating the best model that can fit the data given. I landed up at 81st position in my first attempt itself using techniques different than those already did and it worked pretty well. We got an rmsle of 0.43489 and top scorer being 0.37620.

In [ ]:
import pandas as pd
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import random
import math
import lightgbm as lgb
# from kats.models.prophet import ProphetModel, ProphetParams
# from kats.consts import TimeSeriesData
# from kats.models.var import VARModel, VARParams
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import math
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.metrics import mean_squared_log_error,mean_squared_error
cur_dir=os.getcwd()

In [73]:
train=pd.read_csv(r"C:\Users\Dell\Documents\projects\time series forecasting\train.csv")
train=train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


## Replacing column names with String type

In [74]:
c=train.date.nunique()
c

1684

In [75]:
holidays=pd.read_csv(r"C:\Users\Dell\Documents\projects\time series forecasting\holidays_events.csv")
holidays=holidays.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
holidays

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


## Replacing column names with String type

In [76]:
conditions=[(holidays['type']=='Holiday')&(holidays['transferred']==False),
            (holidays['type']=='Holiday')&(holidays['transferred']==True),
            (holidays['type']=='Transfer')&(holidays['transferred']==False),
            (holidays['type']=='Additional'),(holidays['type']=='Bridge'),
            (holidays['type']=='Event'),(holidays['type']=='Work Day')]
results=['No','Yes','No','No','No','No','Yes']
holidays['work day'] = np.select(conditions, results)
print(holidays)
print(holidays.size)
print(holidays.isnull().count())
print(holidays['work day'].unique())

           date        type    locale locale_name  \
0    2012-03-02     Holiday     Local       Manta   
1    2012-04-01     Holiday  Regional    Cotopaxi   
2    2012-04-12     Holiday     Local      Cuenca   
3    2012-04-14     Holiday     Local    Libertad   
4    2012-04-21     Holiday     Local    Riobamba   
..          ...         ...       ...         ...   
345  2017-12-22  Additional  National     Ecuador   
346  2017-12-23  Additional  National     Ecuador   
347  2017-12-24  Additional  National     Ecuador   
348  2017-12-25     Holiday  National     Ecuador   
349  2017-12-26  Additional  National     Ecuador   

                       description  transferred work day  
0               Fundacion de Manta        False       No  
1    Provincializacion de Cotopaxi        False       No  
2              Fundacion de Cuenca        False       No  
3        Cantonizacion de Libertad        False       No  
4        Cantonizacion de Riobamba        False       No  
..       

In [77]:
oil=pd.read_csv(r"C:\Users\Dell\Documents\projects\time series forecasting\oil.csv")
oil=oil.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


## Replacing column names with String type

In [78]:
sample_submission=pd.read_csv(r"C:\Users\Dell\Documents\projects\time series forecasting\sample_submission.csv")
sample_submission.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
sample_submission

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0
...,...,...
28507,3029395,0.0
28508,3029396,0.0
28509,3029397,0.0
28510,3029398,0.0


## Replacing column names with String type

In [79]:
test=pd.read_csv(r"C:\Users\Dell\Documents\projects\time series forecasting\test.csv")
test=test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test


,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


## Replacing column names with String type

In [80]:
train=pd.merge(train,holidays, on='date', how='left')

## merging holidays data with train.

In [81]:
train

,id,date,store_nbr,family,sales,onpromotion,type,locale,locale_name,description,transferred,work day
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,No
1,1,2013-01-01,1,BABY CARE,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,No
2,2,2013-01-01,1,BEAUTY,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,No
3,3,2013-01-01,1,BEVERAGES,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,No
4,4,2013-01-01,1,BOOKS,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,No
...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,No
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Holiday,Local,Riobamba,Fundacion de Riobamba,False,No
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Holiday,Local,Riobamba,Fundacion de Riobamba,False,No
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Holiday,Local,Riobamba,Fundacion de Riobamba,False,No


In [82]:
oil.rename(columns={'dcoilwtico':'oil prices'},inplace=True)

In [83]:
oil

,date,oil prices
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [84]:
mean_oil=oil['oil prices'].mean(skipna=True)
mean_oil

67.71436595744682

In [85]:
oil['oil prices'].fillna(mean_oil)

0       67.714366
1       93.140000
2       92.970000
3       93.120000
4       93.200000
          ...    
1213    47.650000
1214    46.400000
1215    46.460000
1216    45.960000
1217    47.260000
Name: oil prices, Length: 1218, dtype: float64

## we're replacing the missing values in oil prices with their mean.

In [86]:
train

,id,date,store_nbr,family,sales,onpromotion,type,locale,locale_name,description,transferred,work day
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,No
1,1,2013-01-01,1,BABY CARE,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,No
2,2,2013-01-01,1,BEAUTY,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,No
3,3,2013-01-01,1,BEVERAGES,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,No
4,4,2013-01-01,1,BOOKS,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,No
...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,No
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Holiday,Local,Riobamba,Fundacion de Riobamba,False,No
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Holiday,Local,Riobamba,Fundacion de Riobamba,False,No
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Holiday,Local,Riobamba,Fundacion de Riobamba,False,No


In [87]:
train=pd.merge(train,oil, on='date', how='left')
train

,id,date,store_nbr,family,sales,onpromotion,type,locale,locale_name,description,transferred,work day,oil prices
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,No,NaN
1,1,2013-01-01,1,BABY CARE,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,No,NaN
2,2,2013-01-01,1,BEAUTY,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,No,NaN
3,3,2013-01-01,1,BEVERAGES,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,No,NaN
4,4,2013-01-01,1,BOOKS,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,No,47.57
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Holiday,Local,Riobamba,Fundacion de Riobamba,False,No,47.57
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Holiday,Local,Riobamba,Fundacion de Riobamba,False,No,47.57
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Holiday,Local,Riobamba,Fundacion de Riobamba,False,No,47.57


## Merging training and oil tables

In [88]:
train=train.drop(columns=['type','locale','locale_name','description','transferred'])

In [89]:
train.isnull().sum()

id                   0
date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
work day       2551824
oil prices      955152
dtype: int64

In [90]:
train['oil prices'].fillna(mean_oil,inplace=True)
train

,id,date,store_nbr,family,sales,onpromotion,work day,oil prices
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,No,67.714366
1,1,2013-01-01,1,BABY CARE,0.000,0,No,67.714366
2,2,2013-01-01,1,BEAUTY,0.000,0,No,67.714366
3,3,2013-01-01,1,BEVERAGES,0.000,0,No,67.714366
4,4,2013-01-01,1,BOOKS,0.000,0,No,67.714366
...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,No,47.570000
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,No,47.570000
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,No,47.570000
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,No,47.570000


## replacing the oil price values in new table due to left join with mean value found earlier than the new one to avoid the saturation of the data towards the new mean.

In [21]:
train

,id,date,store_nbr,family,sales,onpromotion,work day,oil prices
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,No,67.714366
1,1,2013-01-01,1,BABY CARE,0.000,0,No,67.714366
2,2,2013-01-01,1,BEAUTY,0.000,0,No,67.714366
3,3,2013-01-01,1,BEVERAGES,0.000,0,No,67.714366
4,4,2013-01-01,1,BOOKS,0.000,0,No,67.714366
...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,No,47.570000
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,No,47.570000
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,No,47.570000
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,No,47.570000


In [22]:
train['work day'].fillna('yes',inplace=True)
train

,id,date,store_nbr,family,sales,onpromotion,work day,oil prices
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,No,67.714366
1,1,2013-01-01,1,BABY CARE,0.000,0,No,67.714366
2,2,2013-01-01,1,BEAUTY,0.000,0,No,67.714366
3,3,2013-01-01,1,BEVERAGES,0.000,0,No,67.714366
4,4,2013-01-01,1,BOOKS,0.000,0,No,67.714366
...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,No,47.570000
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,No,47.570000
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,No,47.570000
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,No,47.570000


## we are replacing the missing holiday values as work day. also in the data there is no mention of store number and the location of each shops of purchase so, we'll assume that the holiday data corresponds to the shop/region where a purchase happened on that day. we'll check with this assumption if it fits well.

In [23]:
train.isnull().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
work day       0
oil prices     0
dtype: int64

## no null values 

In [24]:
train['date']=pd.to_datetime(train['date'])

In [25]:
train

,id,date,store_nbr,family,sales,onpromotion,work day,oil prices
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,No,67.714366
1,1,2013-01-01,1,BABY CARE,0.000,0,No,67.714366
2,2,2013-01-01,1,BEAUTY,0.000,0,No,67.714366
3,3,2013-01-01,1,BEVERAGES,0.000,0,No,67.714366
4,4,2013-01-01,1,BOOKS,0.000,0,No,67.714366
...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,No,47.570000
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,No,47.570000
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,No,47.570000
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,No,47.570000


In [26]:
train['date'] = train.date.dt.to_period('D')

In [27]:
train

,id,date,store_nbr,family,sales,onpromotion,work day,oil prices
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,No,67.714366
1,1,2013-01-01,1,BABY CARE,0.000,0,No,67.714366
2,2,2013-01-01,1,BEAUTY,0.000,0,No,67.714366
3,3,2013-01-01,1,BEVERAGES,0.000,0,No,67.714366
4,4,2013-01-01,1,BOOKS,0.000,0,No,67.714366
...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,No,47.570000
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,No,47.570000
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,No,47.570000
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,No,47.570000


In [28]:
new=train.copy()

In [29]:
new

,id,date,store_nbr,family,sales,onpromotion,work day,oil prices
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,No,67.714366
1,1,2013-01-01,1,BABY CARE,0.000,0,No,67.714366
2,2,2013-01-01,1,BEAUTY,0.000,0,No,67.714366
3,3,2013-01-01,1,BEVERAGES,0.000,0,No,67.714366
4,4,2013-01-01,1,BOOKS,0.000,0,No,67.714366
...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,No,47.570000
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,No,47.570000
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,No,47.570000
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,No,47.570000


In [70]:
fourier = CalendarFourier(freq='M', order=4)
dp = DeterministicProcess(
    index=new['date'].unique(),
    constant=True,
    order=1,
    seasonal=True, 
    additional_terms=[fourier],
    drop=True,
)
X=dp.in_sample()

## we're converting the dates into deterministic terms so that we can use it for prediction. we'll have a constant upward trend, seasonal terms with periodicity of 7 to represent the days and we'll use a  fourier series of order 4 to represent the monthly periodic component of the dates.

In [71]:
X

,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)","sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)","sin(4,freq=M)","cos(4,freq=M)"
2013-01-01,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
2013-01-02,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.201299,0.979530,0.394356,0.918958,0.571268,0.820763,0.724793,0.688967
2013-01-03,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.394356,0.918958,0.724793,0.688967,0.937752,0.347305,0.998717,-0.050649
2013-01-04,1.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.571268,0.820763,0.937752,0.347305,0.968077,-0.250653,0.651372,-0.758758
2013-01-05,1.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.724793,0.688967,0.998717,-0.050649,0.651372,-0.758758,-0.101168,-0.994869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-11,1.0,1680.0,0.0,0.0,0.0,0.0,0.0,1.0,0.897805,-0.440394,-0.790776,-0.612106,-0.201299,0.979530,0.968077,-0.250653
2017-08-12,1.0,1681.0,0.0,0.0,0.0,0.0,0.0,0.0,0.790776,-0.612106,-0.968077,-0.250653,0.394356,0.918958,0.485302,-0.874347
2017-08-13,1.0,1682.0,1.0,0.0,0.0,0.0,0.0,0.0,0.651372,-0.758758,-0.988468,0.151428,0.848644,0.528964,-0.299363,-0.954139
2017-08-14,1.0,1683.0,0.0,1.0,0.0,0.0,0.0,0.0,0.485302,-0.874347,-0.848644,0.528964,0.998717,-0.050649,-0.897805,-0.440394


In [46]:
print(fourier)

Fourier(freq=M, order=4)


In [33]:
X.reset_index(inplace=True)
X

,index,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)"
0,2013-01-01,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2013-01-02,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2013-01-03,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0
3,2013-01-04,1.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2013-01-05,1.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
1679,2017-08-11,1.0,1680.0,0.0,0.0,0.0,0.0,0.0,1.0
1680,2017-08-12,1.0,1681.0,0.0,0.0,0.0,0.0,0.0,0.0
1681,2017-08-13,1.0,1682.0,1.0,0.0,0.0,0.0,0.0,0.0
1682,2017-08-14,1.0,1683.0,0.0,1.0,0.0,0.0,0.0,0.0


In [34]:
X.rename(columns={'index':'date'},inplace=True)

In [35]:
X

,date,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)","sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)","sin(4,freq=M)","cos(4,freq=M)"
0,2013-01-01,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
1,2013-01-02,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.201299,0.979530,0.394356,0.918958,0.571268,0.820763,0.724793,0.688967
2,2013-01-03,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.394356,0.918958,0.724793,0.688967,0.937752,0.347305,0.998717,-0.050649
3,2013-01-04,1.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.571268,0.820763,0.937752,0.347305,0.968077,-0.250653,0.651372,-0.758758
4,2013-01-05,1.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.724793,0.688967,0.998717,-0.050649,0.651372,-0.758758,-0.101168,-0.994869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,2017-08-11,1.0,1680.0,0.0,0.0,0.0,0.0,0.0,1.0,0.897805,-0.440394,-0.790776,-0.612106,-0.201299,0.979530,0.968077,-0.250653
1680,2017-08-12,1.0,1681.0,0.0,0.0,0.0,0.0,0.0,0.0,0.790776,-0.612106,-0.968077,-0.250653,0.394356,0.918958,0.485302,-0.874347
1681,2017-08-13,1.0,1682.0,1.0,0.0,0.0,0.0,0.0,0.0,0.651372,-0.758758,-0.988468,0.151428,0.848644,0.528964,-0.299363,-0.954139
1682,2017-08-14,1.0,1683.0,0.0,1.0,0.0,0.0,0.0,0.0,0.485302,-0.874347,-0.848644,0.528964,0.998717,-0.050649,-0.897805,-0.440394


In [36]:
new=pd.merge(new,X,on='date',how='left')

## we're replacing the dates with our new deterministic terms in the train data.

In [37]:
new.isnull().sum()

id               0
date             0
store_nbr        0
family           0
sales            0
onpromotion      0
work day         0
oil prices       0
const            0
trend            0
s(2,7)           0
s(3,7)           0
s(4,7)           0
s(5,7)           0
s(6,7)           0
s(7,7)           0
sin(1,freq=M)    0
cos(1,freq=M)    0
sin(2,freq=M)    0
cos(2,freq=M)    0
sin(3,freq=M)    0
cos(3,freq=M)    0
sin(4,freq=M)    0
cos(4,freq=M)    0
dtype: int64

In [38]:
new

,id,date,store_nbr,family,sales,onpromotion,work day,oil prices,const,trend,...,"s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)","sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)","sin(4,freq=M)","cos(4,freq=M)"
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,No,67.714366,1.0,1.0,...,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
1,1,2013-01-01,1,BABY CARE,0.000,0,No,67.714366,1.0,1.0,...,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
2,2,2013-01-01,1,BEAUTY,0.000,0,No,67.714366,1.0,1.0,...,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
3,3,2013-01-01,1,BEVERAGES,0.000,0,No,67.714366,1.0,1.0,...,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
4,4,2013-01-01,1,BOOKS,0.000,0,No,67.714366,1.0,1.0,...,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,No,47.570000,1.0,1684.0,...,0.0,0.0,0.299363,-0.954139,-0.571268,0.820763,0.790776,-0.612106,-0.937752,0.347305
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,No,47.570000,1.0,1684.0,...,0.0,0.0,0.299363,-0.954139,-0.571268,0.820763,0.790776,-0.612106,-0.937752,0.347305
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,No,47.570000,1.0,1684.0,...,0.0,0.0,0.299363,-0.954139,-0.571268,0.820763,0.790776,-0.612106,-0.937752,0.347305
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,No,47.570000,1.0,1684.0,...,0.0,0.0,0.299363,-0.954139,-0.571268,0.820763,0.790776,-0.612106,-0.937752,0.347305


In [39]:
family_list=set(train['family'].to_list())

In [40]:
len(family_list)

33

In [41]:
store_no=set(train['store_nbr'].to_list())
len(store_no)

54

In [42]:
for ele in family_list:
    new[ele]=np.where(new['family']==ele,1,0)
for h in store_no:
    new[h]=np.where(new['store_nbr']==h,1,0)

## one-hot encoding all the categorical columns.(there's no ordinal columns)

In [43]:
new

,id,date,store_nbr,family,sales,onpromotion,work day,oil prices,const,trend,...,45,46,47,48,49,50,51,52,53,54
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,No,67.714366,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,1,2013-01-01,1,BABY CARE,0.000,0,No,67.714366,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,2,2013-01-01,1,BEAUTY,0.000,0,No,67.714366,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,3,2013-01-01,1,BEVERAGES,0.000,0,No,67.714366,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,4,2013-01-01,1,BOOKS,0.000,0,No,67.714366,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,No,47.570000,1.0,1684.0,...,0,0,0,0,0,0,0,0,0,0
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,No,47.570000,1.0,1684.0,...,0,0,0,0,0,0,0,0,0,0
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,No,47.570000,1.0,1684.0,...,0,0,0,0,0,0,0,0,0,0
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,No,47.570000,1.0,1684.0,...,0,0,0,0,0,0,0,0,0,0


# converting the test data to the format of cleaned train data with similar steps

In [44]:
test=pd.merge(test,oil,on='date',how='left')

In [45]:
test

,id,date,store_nbr,family,onpromotion,oil prices
0,3000888,2017-08-16,1,AUTOMOTIVE,0,46.80
1,3000889,2017-08-16,1,BABY CARE,0,46.80
2,3000890,2017-08-16,1,BEAUTY,2,46.80
3,3000891,2017-08-16,1,BEVERAGES,20,46.80
4,3000892,2017-08-16,1,BOOKS,0,46.80
...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,47.26
28508,3029396,2017-08-31,9,PREPARED FOODS,0,47.26
28509,3029397,2017-08-31,9,PRODUCE,1,47.26
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,47.26


In [46]:
test.isnull().sum()

id                0
date              0
store_nbr         0
family            0
onpromotion       0
oil prices     7128
dtype: int64

In [47]:
test['oil prices']=test['oil prices'].fillna(mean_oil)

## changing the missing oil prices

In [48]:
test.isnull().sum()

id             0
date           0
store_nbr      0
family         0
onpromotion    0
oil prices     0
dtype: int64

In [49]:
test=pd.merge(test,holidays,on='date',how='left')
test.isnull().sum()

id                 0
date               0
store_nbr          0
family             0
onpromotion        0
oil prices         0
type           26730
locale         26730
locale_name    26730
description    26730
transferred    26730
work day       26730
dtype: int64

## merging test and oil tables

In [50]:
test['work day']=test['work day'].fillna('yes')
test.isnull().sum()

id                 0
date               0
store_nbr          0
family             0
onpromotion        0
oil prices         0
type           26730
locale         26730
locale_name    26730
description    26730
transferred    26730
work day           0
dtype: int64

## we're replacing all the values which did'nt have mention about the holidays as work day. Also going with the similar assumption we used on train data.

In [51]:
test=test.drop(columns=['type','locale','locale_name','description','transferred'])

In [52]:
test

,id,date,store_nbr,family,onpromotion,oil prices,work day
0,3000888,2017-08-16,1,AUTOMOTIVE,0,46.80,yes
1,3000889,2017-08-16,1,BABY CARE,0,46.80,yes
2,3000890,2017-08-16,1,BEAUTY,2,46.80,yes
3,3000891,2017-08-16,1,BEVERAGES,20,46.80,yes
4,3000892,2017-08-16,1,BOOKS,0,46.80,yes
...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,47.26,yes
28508,3029396,2017-08-31,9,PREPARED FOODS,0,47.26,yes
28509,3029397,2017-08-31,9,PRODUCE,1,47.26,yes
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,47.26,yes


In [92]:
a=set(train['family'].to_list())

In [93]:
len(a)

33

In [94]:
test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [96]:
for ele in a:
    test[ele]=np.where(test['family']==ele,1,0)

In [57]:
test

,id,date,store_nbr,family,onpromotion,oil prices,work day,PET SUPPLIES,CLEANING,HOME APPLIANCES,...,"LIQUOR,WINE,BEER",PRODUCE,EGGS,BEVERAGES,HOME AND KITCHEN II,PERSONAL CARE,BEAUTY,HARDWARE,BOOKS,LINGERIE
0,3000888,2017-08-16,1,AUTOMOTIVE,0,46.80,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3000889,2017-08-16,1,BABY CARE,0,46.80,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3000890,2017-08-16,1,BEAUTY,2,46.80,yes,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,3000891,2017-08-16,1,BEVERAGES,20,46.80,yes,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,3000892,2017-08-16,1,BOOKS,0,46.80,yes,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,47.26,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28508,3029396,2017-08-31,9,PREPARED FOODS,0,47.26,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28509,3029397,2017-08-31,9,PRODUCE,1,47.26,yes,0,0,0,...,0,1,0,0,0,0,0,0,0,0
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,47.26,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
g=set(test['store_nbr'].to_list())

In [59]:
print(g)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54}


In [ ]:
for ele in g:
    test[ele]=np.where(test['store_nbr']==ele,1,0)

In [61]:
test

,id,date,store_nbr,family,onpromotion,oil prices,work day,PET SUPPLIES,CLEANING,HOME APPLIANCES,...,45,46,47,48,49,50,51,52,53,54
0,3000888,2017-08-16,1,AUTOMOTIVE,0,46.80,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3000889,2017-08-16,1,BABY CARE,0,46.80,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3000890,2017-08-16,1,BEAUTY,2,46.80,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3000891,2017-08-16,1,BEVERAGES,20,46.80,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3000892,2017-08-16,1,BOOKS,0,46.80,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,47.26,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28508,3029396,2017-08-31,9,PREPARED FOODS,0,47.26,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28509,3029397,2017-08-31,9,PRODUCE,1,47.26,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,47.26,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## In the above steps we one hot encoded the categorical variables in test data.

In [62]:
test_col=list(test.columns.values)
train_col=list(new.columns.values)
print(test_col)

['id', 'date', 'store_nbr', 'family', 'onpromotion', 'oil prices', 'work day', 'PET SUPPLIES', 'CLEANING', 'HOME APPLIANCES', 'DELI', 'POULTRY', 'GROCERY II', 'MAGAZINES', 'GROCERY I', 'DAIRY', 'BABY CARE', 'PLAYERS AND ELECTRONICS', 'AUTOMOTIVE', 'BREAD/BAKERY', 'HOME AND KITCHEN I', 'CELEBRATION', 'MEATS', 'PREPARED FOODS', 'FROZEN FOODS', 'LADIESWEAR', 'LAWN AND GARDEN', 'SCHOOL AND OFFICE SUPPLIES', 'HOME CARE', 'SEAFOOD', 'LIQUOR,WINE,BEER', 'PRODUCE', 'EGGS', 'BEVERAGES', 'HOME AND KITCHEN II', 'PERSONAL CARE', 'BEAUTY', 'HARDWARE', 'BOOKS', 'LINGERIE', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]


In [63]:
print(train_col)

['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion', 'work day', 'oil prices', 'const', 'trend', 's(2,7)', 's(3,7)', 's(4,7)', 's(5,7)', 's(6,7)', 's(7,7)', 'sin(1,freq=M)', 'cos(1,freq=M)', 'sin(2,freq=M)', 'cos(2,freq=M)', 'sin(3,freq=M)', 'cos(3,freq=M)', 'sin(4,freq=M)', 'cos(4,freq=M)', 'PET SUPPLIES', 'CLEANING', 'HOME APPLIANCES', 'DELI', 'POULTRY', 'GROCERY II', 'MAGAZINES', 'GROCERY I', 'DAIRY', 'BABY CARE', 'PLAYERS AND ELECTRONICS', 'AUTOMOTIVE', 'BREAD/BAKERY', 'HOME AND KITCHEN I', 'CELEBRATION', 'MEATS', 'PREPARED FOODS', 'FROZEN FOODS', 'LADIESWEAR', 'LAWN AND GARDEN', 'SCHOOL AND OFFICE SUPPLIES', 'HOME CARE', 'SEAFOOD', 'LIQUOR,WINE,BEER', 'PRODUCE', 'EGGS', 'BEVERAGES', 'HOME AND KITCHEN II', 'PERSONAL CARE', 'BEAUTY', 'HARDWARE', 'BOOKS', 'LINGERIE', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]


In [64]:
test['date']=pd.to_datetime(test['date'])

In [65]:
test['date'] = test.date.dt.to_period('D')

In [66]:
test

,id,date,store_nbr,family,onpromotion,oil prices,work day,PET SUPPLIES,CLEANING,HOME APPLIANCES,...,45,46,47,48,49,50,51,52,53,54
0,3000888,2017-08-16,1,AUTOMOTIVE,0,46.80,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3000889,2017-08-16,1,BABY CARE,0,46.80,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3000890,2017-08-16,1,BEAUTY,2,46.80,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3000891,2017-08-16,1,BEVERAGES,20,46.80,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3000892,2017-08-16,1,BOOKS,0,46.80,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,47.26,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28508,3029396,2017-08-31,9,PREPARED FOODS,0,47.26,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28509,3029397,2017-08-31,9,PRODUCE,1,47.26,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,47.26,yes,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
T_X=dp.out_of_sample(steps=16)

## Finding the deterministeic terms for next 16 days

In [68]:
T_X

,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)","sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)","sin(4,freq=M)","cos(4,freq=M)"
2017-08-16,1.0,1685.0,0.0,0.0,0.0,1.0,0.0,0.0,0.101168,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958
2017-08-17,1.0,1686.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.101168,-0.994869,0.201299,0.979530,-0.299363,-0.954139,0.394356,0.918958
2017-08-18,1.0,1687.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.299363,-0.954139,0.571268,0.820763,-0.790776,-0.612106,0.937752,0.347305
2017-08-19,1.0,1688.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.485302,-0.874347,0.848644,0.528964,-0.998717,-0.050649,0.897805,-0.440394
2017-08-20,1.0,1689.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.651372,-0.758758,0.988468,0.151428,-0.848644,0.528964,0.299363,-0.954139
2017-08-21,1.0,1690.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.790776,-0.612106,0.968077,-0.250653,-0.394356,0.918958,-0.485302,-0.874347
2017-08-22,1.0,1691.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.897805,-0.440394,0.790776,-0.612106,0.201299,0.979530,-0.968077,-0.250653
2017-08-23,1.0,1692.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.968077,-0.250653,0.485302,-0.874347,0.724793,0.688967,-0.848644,0.528964
2017-08-24,1.0,1693.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.998717,-0.050649,0.101168,-0.994869,0.988468,0.151428,-0.201299,0.979530
2017-08-25,1.0,1694.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.988468,0.151428,-0.299363,-0.954139,0.897805,-0.440394,0.571268,0.820763


In [69]:
T_X.reset_index(inplace=True)

In [70]:
T_X.rename(columns={'index':'date'},inplace=True)

In [71]:
test=pd.merge(test,T_X,on='date',how='left')

In [72]:
test

,id,date,store_nbr,family,onpromotion,oil prices,work day,PET SUPPLIES,CLEANING,HOME APPLIANCES,...,"s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)","sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)","sin(4,freq=M)","cos(4,freq=M)"
0,3000888,2017-08-16,1,AUTOMOTIVE,0,46.80,yes,0,0,0,...,0.0,0.0,0.101168,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958
1,3000889,2017-08-16,1,BABY CARE,0,46.80,yes,0,0,0,...,0.0,0.0,0.101168,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958
2,3000890,2017-08-16,1,BEAUTY,2,46.80,yes,0,0,0,...,0.0,0.0,0.101168,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958
3,3000891,2017-08-16,1,BEVERAGES,20,46.80,yes,0,0,0,...,0.0,0.0,0.101168,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958
4,3000892,2017-08-16,1,BOOKS,0,46.80,yes,0,0,0,...,0.0,0.0,0.101168,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,47.26,yes,0,0,0,...,1.0,0.0,-0.201299,0.979530,-0.394356,0.918958,-0.571268,0.820763,-0.724793,0.688967
28508,3029396,2017-08-31,9,PREPARED FOODS,0,47.26,yes,0,0,0,...,1.0,0.0,-0.201299,0.979530,-0.394356,0.918958,-0.571268,0.820763,-0.724793,0.688967
28509,3029397,2017-08-31,9,PRODUCE,1,47.26,yes,0,0,0,...,1.0,0.0,-0.201299,0.979530,-0.394356,0.918958,-0.571268,0.820763,-0.724793,0.688967
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,47.26,yes,0,0,0,...,1.0,0.0,-0.201299,0.979530,-0.394356,0.918958,-0.571268,0.820763,-0.724793,0.688967


In [73]:
Y=new['sales'].copy()

In [74]:
Y

0             0.000
1             0.000
2             0.000
3             0.000
4             0.000
             ...   
3054343     438.133
3054344     154.553
3054345    2419.729
3054346     121.000
3054347      16.000
Name: sales, Length: 3054348, dtype: float64

In [75]:
new=new.drop(columns=['date','store_nbr','family'])

In [76]:
new

,id,sales,onpromotion,work day,oil prices,const,trend,"s(2,7)","s(3,7)","s(4,7)",...,45,46,47,48,49,50,51,52,53,54
0,0,0.000,0,No,67.714366,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.000,0,No,67.714366,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2,0.000,0,No,67.714366,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,3,0.000,0,No,67.714366,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,4,0.000,0,No,67.714366,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,438.133,0,No,47.570000,1.0,1684.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3054344,3000884,154.553,1,No,47.570000,1.0,1684.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3054345,3000885,2419.729,148,No,47.570000,1.0,1684.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3054346,3000886,121.000,8,No,47.570000,1.0,1684.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
a=set(list(new['work day'].unique()))
a=list(a)
a

['No', 'yes', 'Yes']

In [78]:
for ele in a:
    new[ele]=np.where(new['work day']==ele,1,0)
    test[ele]=np.where(test['work day']==ele,1,0)
    

In [79]:
b=list(test.columns.values)
b

['id',
 'date',
 'store_nbr',
 'family',
 'onpromotion',
 'oil prices',
 'work day',
 'PET SUPPLIES',
 'CLEANING',
 'HOME APPLIANCES',
 'DELI',
 'POULTRY',
 'GROCERY II',
 'MAGAZINES',
 'GROCERY I',
 'DAIRY',
 'BABY CARE',
 'PLAYERS AND ELECTRONICS',
 'AUTOMOTIVE',
 'BREAD/BAKERY',
 'HOME AND KITCHEN I',
 'CELEBRATION',
 'MEATS',
 'PREPARED FOODS',
 'FROZEN FOODS',
 'LADIESWEAR',
 'LAWN AND GARDEN',
 'SCHOOL AND OFFICE SUPPLIES',
 'HOME CARE',
 'SEAFOOD',
 'LIQUOR,WINE,BEER',
 'PRODUCE',
 'EGGS',
 'BEVERAGES',
 'HOME AND KITCHEN II',
 'PERSONAL CARE',
 'BEAUTY',
 'HARDWARE',
 'BOOKS',
 'LINGERIE',
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 'const',
 'trend',
 's(2,7)',
 's(3,7)',
 's(4,7)',
 's(5,7)',
 's(6,7)',
 's(7,7)',
 'sin(1,freq=M)',
 'cos(1,freq=M)',
 'sin(2,freq

In [80]:
test=test.drop(columns=['date','store_nbr','family'])
test

,id,onpromotion,oil prices,work day,PET SUPPLIES,CLEANING,HOME APPLIANCES,DELI,POULTRY,GROCERY II,...,"cos(1,freq=M)","sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)","sin(4,freq=M)","cos(4,freq=M)",No,yes,Yes
0,3000888,0,46.80,yes,0,0,0,0,0,0,...,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958,0,1,0
1,3000889,0,46.80,yes,0,0,0,0,0,0,...,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958,0,1,0
2,3000890,2,46.80,yes,0,0,0,0,0,0,...,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958,0,1,0
3,3000891,20,46.80,yes,0,0,0,0,0,0,...,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958,0,1,0
4,3000892,0,46.80,yes,0,0,0,0,0,0,...,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,1,47.26,yes,0,0,0,0,1,0,...,0.979530,-0.394356,0.918958,-0.571268,0.820763,-0.724793,0.688967,0,1,0
28508,3029396,0,47.26,yes,0,0,0,0,0,0,...,0.979530,-0.394356,0.918958,-0.571268,0.820763,-0.724793,0.688967,0,1,0
28509,3029397,1,47.26,yes,0,0,0,0,0,0,...,0.979530,-0.394356,0.918958,-0.571268,0.820763,-0.724793,0.688967,0,1,0
28510,3029398,9,47.26,yes,0,0,0,0,0,0,...,0.979530,-0.394356,0.918958,-0.571268,0.820763,-0.724793,0.688967,0,1,0


In [81]:
a=set(list(test['work day'].unique()))
a

{'No', 'yes'}

In [82]:
new.drop(columns=['sales'],inplace=True)

In [83]:
new.drop(columns=['work day'],inplace=True)
test.drop(columns=['work day'],inplace=True)

In [84]:
new

,id,onpromotion,oil prices,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)",...,48,49,50,51,52,53,54,No,yes,Yes
0,0,0,67.714366,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,67.714366,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
2,2,0,67.714366,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3,3,0,67.714366,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,4,0,67.714366,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,0,47.570000,1.0,1684.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3054344,3000884,1,47.570000,1.0,1684.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3054345,3000885,148,47.570000,1.0,1684.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3054346,3000886,8,47.570000,1.0,1684.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0


In [85]:
test

,id,onpromotion,oil prices,PET SUPPLIES,CLEANING,HOME APPLIANCES,DELI,POULTRY,GROCERY II,MAGAZINES,...,"cos(1,freq=M)","sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)","sin(4,freq=M)","cos(4,freq=M)",No,yes,Yes
0,3000888,0,46.80,0,0,0,0,0,0,0,...,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958,0,1,0
1,3000889,0,46.80,0,0,0,0,0,0,0,...,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958,0,1,0
2,3000890,2,46.80,0,0,0,0,0,0,0,...,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958,0,1,0
3,3000891,20,46.80,0,0,0,0,0,0,0,...,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958,0,1,0
4,3000892,0,46.80,0,0,0,0,0,0,0,...,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,1,47.26,0,0,0,0,1,0,0,...,0.979530,-0.394356,0.918958,-0.571268,0.820763,-0.724793,0.688967,0,1,0
28508,3029396,0,47.26,0,0,0,0,0,0,0,...,0.979530,-0.394356,0.918958,-0.571268,0.820763,-0.724793,0.688967,0,1,0
28509,3029397,1,47.26,0,0,0,0,0,0,0,...,0.979530,-0.394356,0.918958,-0.571268,0.820763,-0.724793,0.688967,0,1,0
28510,3029398,9,47.26,0,0,0,0,0,0,0,...,0.979530,-0.394356,0.918958,-0.571268,0.820763,-0.724793,0.688967,0,1,0


In [86]:
new.columns=new.columns.astype(str)
test.columns=test.columns.astype(str)

In [87]:
new_col=list(new.columns)
new_col

['id',
 'onpromotion',
 'oil prices',
 'const',
 'trend',
 's(2,7)',
 's(3,7)',
 's(4,7)',
 's(5,7)',
 's(6,7)',
 's(7,7)',
 'sin(1,freq=M)',
 'cos(1,freq=M)',
 'sin(2,freq=M)',
 'cos(2,freq=M)',
 'sin(3,freq=M)',
 'cos(3,freq=M)',
 'sin(4,freq=M)',
 'cos(4,freq=M)',
 'PET SUPPLIES',
 'CLEANING',
 'HOME APPLIANCES',
 'DELI',
 'POULTRY',
 'GROCERY II',
 'MAGAZINES',
 'GROCERY I',
 'DAIRY',
 'BABY CARE',
 'PLAYERS AND ELECTRONICS',
 'AUTOMOTIVE',
 'BREAD/BAKERY',
 'HOME AND KITCHEN I',
 'CELEBRATION',
 'MEATS',
 'PREPARED FOODS',
 'FROZEN FOODS',
 'LADIESWEAR',
 'LAWN AND GARDEN',
 'SCHOOL AND OFFICE SUPPLIES',
 'HOME CARE',
 'SEAFOOD',
 'LIQUOR,WINE,BEER',
 'PRODUCE',
 'EGGS',
 'BEVERAGES',
 'HOME AND KITCHEN II',
 'PERSONAL CARE',
 'BEAUTY',
 'HARDWARE',
 'BOOKS',
 'LINGERIE',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32

In [88]:
test_col=list(test.columns)
test_col

['id',
 'onpromotion',
 'oil prices',
 'PET SUPPLIES',
 'CLEANING',
 'HOME APPLIANCES',
 'DELI',
 'POULTRY',
 'GROCERY II',
 'MAGAZINES',
 'GROCERY I',
 'DAIRY',
 'BABY CARE',
 'PLAYERS AND ELECTRONICS',
 'AUTOMOTIVE',
 'BREAD/BAKERY',
 'HOME AND KITCHEN I',
 'CELEBRATION',
 'MEATS',
 'PREPARED FOODS',
 'FROZEN FOODS',
 'LADIESWEAR',
 'LAWN AND GARDEN',
 'SCHOOL AND OFFICE SUPPLIES',
 'HOME CARE',
 'SEAFOOD',
 'LIQUOR,WINE,BEER',
 'PRODUCE',
 'EGGS',
 'BEVERAGES',
 'HOME AND KITCHEN II',
 'PERSONAL CARE',
 'BEAUTY',
 'HARDWARE',
 'BOOKS',
 'LINGERIE',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 'const',
 'trend',
 's(2,7)',
 's(3,7)',
 's(4,7)',
 's(5,7)',
 's(6,7)'

In [89]:
new=new.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test=test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [90]:
new_col=set(list(new.columns))
test_col=set(list(test.columns))
d=test_col-new_col
d

set()

In [91]:
new_col=list(new.columns)
test_col=list(test_1.columns)
#d=test_col-new_col
if new_col==test_col:
    print(True)
print(7)    

NameError: name 'test_1' is not defined

In [92]:
new_col=list(new.columns)
new_col

['id',
 'onpromotion',
 'oilprices',
 'const',
 'trend',
 's27',
 's37',
 's47',
 's57',
 's67',
 's77',
 'sin1freqM',
 'cos1freqM',
 'sin2freqM',
 'cos2freqM',
 'sin3freqM',
 'cos3freqM',
 'sin4freqM',
 'cos4freqM',
 'PETSUPPLIES',
 'CLEANING',
 'HOMEAPPLIANCES',
 'DELI',
 'POULTRY',
 'GROCERYII',
 'MAGAZINES',
 'GROCERYI',
 'DAIRY',
 'BABYCARE',
 'PLAYERSANDELECTRONICS',
 'AUTOMOTIVE',
 'BREADBAKERY',
 'HOMEANDKITCHENI',
 'CELEBRATION',
 'MEATS',
 'PREPAREDFOODS',
 'FROZENFOODS',
 'LADIESWEAR',
 'LAWNANDGARDEN',
 'SCHOOLANDOFFICESUPPLIES',
 'HOMECARE',
 'SEAFOOD',
 'LIQUORWINEBEER',
 'PRODUCE',
 'EGGS',
 'BEVERAGES',
 'HOMEANDKITCHENII',
 'PERSONALCARE',
 'BEAUTY',
 'HARDWARE',
 'BOOKS',
 'LINGERIE',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '4

In [93]:
column_names=['id', 'onpromotion', 'oil prices', 'const', 'trend', 's(2,7)', 's(3,7)', 's(4,7)', 's(5,7)', 's(6,7)', 's(7,7)', 'sin(1,freq=M)', 'cos(1,freq=M)', 'sin(2,freq=M)', 'cos(2,freq=M)', 'sin(3,freq=M)', 'cos(3,freq=M)', 'sin(4,freq=M)', 'cos(4,freq=M)', 'HOME AND KITCHEN II', 'HOME APPLIANCES', 'CELEBRATION', 'BEAUTY', 'DELI', 'PRODUCE', 'FROZEN FOODS', 'GROCERY II', 'EGGS', 'LIQUOR,WINE,BEER', 'LAWN AND GARDEN', 'DAIRY', 'HARDWARE', 'PET SUPPLIES', 'PREPARED FOODS', 'GROCERY I', 'SCHOOL AND OFFICE SUPPLIES', 'HOME CARE', 'MEATS', 'HOME AND KITCHEN I', 'PERSONAL CARE', 'PLAYERS AND ELECTRONICS', 'BOOKS', 'MAGAZINES', 'CLEANING', 'BABY CARE', 'BEVERAGES', 'LINGERIE', 'BREAD/BAKERY', 'SEAFOOD', 'POULTRY', 'LADIESWEAR', 'AUTOMOTIVE', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', 'yes', 'Yes', 'No']

In [94]:
test_1=test.loc[:,new_col].copy()

In [95]:
test_1

,id,onpromotion,oilprices,const,trend,s27,s37,s47,s57,s67,...,48,49,50,51,52,53,54,No,yes,Yes
0,3000888,0,46.80,1.0,1685.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,0
1,3000889,0,46.80,1.0,1685.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2,3000890,2,46.80,1.0,1685.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3,3000891,20,46.80,1.0,1685.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,0
4,3000892,0,46.80,1.0,1685.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,1,47.26,1.0,1700.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,1,0
28508,3029396,0,47.26,1.0,1700.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,1,0
28509,3029397,1,47.26,1.0,1700.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,1,0
28510,3029398,9,47.26,1.0,1700.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,1,0


In [96]:
test1_col=list(test_1.columns)
test1_col

['id',
 'onpromotion',
 'oilprices',
 'const',
 'trend',
 's27',
 's37',
 's47',
 's57',
 's67',
 's77',
 'sin1freqM',
 'cos1freqM',
 'sin2freqM',
 'cos2freqM',
 'sin3freqM',
 'cos3freqM',
 'sin4freqM',
 'cos4freqM',
 'PETSUPPLIES',
 'CLEANING',
 'HOMEAPPLIANCES',
 'DELI',
 'POULTRY',
 'GROCERYII',
 'MAGAZINES',
 'GROCERYI',
 'DAIRY',
 'BABYCARE',
 'PLAYERSANDELECTRONICS',
 'AUTOMOTIVE',
 'BREADBAKERY',
 'HOMEANDKITCHENI',
 'CELEBRATION',
 'MEATS',
 'PREPAREDFOODS',
 'FROZENFOODS',
 'LADIESWEAR',
 'LAWNANDGARDEN',
 'SCHOOLANDOFFICESUPPLIES',
 'HOMECARE',
 'SEAFOOD',
 'LIQUORWINEBEER',
 'PRODUCE',
 'EGGS',
 'BEVERAGES',
 'HOMEANDKITCHENII',
 'PERSONALCARE',
 'BEAUTY',
 'HARDWARE',
 'BOOKS',
 'LINGERIE',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '4

In [97]:
st_scaler=StandardScaler()
temp_st=st_scaler.fit(new)
new_st=temp_st.transform(new)
test_st=temp_st.transform(test_1)

In [98]:
scaler=MinMaxScaler()
temp=scaler.fit(new_st)
new_mm=temp.transform(new_st)
test_mm=temp.transform(test_st)

In [99]:
new_mm

array([[0.00000000e+00, 0.00000000e+00, 4.91820040e-01, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.33234807e-07, 0.00000000e+00, 4.91820040e-01, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.66469614e-07, 0.00000000e+00, 4.91820040e-01, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [9.99999334e-01, 1.99730094e-01, 2.53227526e-01, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.99999667e-01, 1.07962213e-02, 2.53227526e-01, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 2.53227526e-01, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [100]:
test_mm

array([[1.00000033, 0.        , 0.24410754, ..., 0.        , 1.        ,
        0.        ],
       [1.00000067, 0.        , 0.24410754, ..., 0.        , 1.        ,
        0.        ],
       [1.000001  , 0.00269906, 0.24410754, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [1.00950052, 0.00134953, 0.24955585, ..., 0.        , 1.        ,
        0.        ],
       [1.00950086, 0.01214575, 0.24955585, ..., 0.        , 1.        ,
        0.        ],
       [1.00950119, 0.        , 0.24955585, ..., 0.        , 1.        ,
        0.        ]])

In [101]:
new_test=new[:3000000]
test_1=test[3000000:]

In [102]:
new_test

,id,onpromotion,oilprices,const,trend,s27,s37,s47,s57,s67,...,48,49,50,51,52,53,54,No,yes,Yes
0,0,0,67.714366,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,67.714366,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
2,2,0,67.714366,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3,3,0,67.714366,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,4,0,67.714366,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2946535,0,67.714366,1.0,1654.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2999996,2946536,1,67.714366,1.0,1654.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2999997,2946537,0,67.714366,1.0,1654.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2999998,2946538,0,67.714366,1.0,1654.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0


In [103]:
test_1

,id,onpromotion,oilprices,PETSUPPLIES,CLEANING,HOMEAPPLIANCES,DELI,POULTRY,GROCERYII,MAGAZINES,...,cos1freqM,sin2freqM,cos2freqM,sin3freqM,cos3freqM,sin4freqM,cos4freqM,No,yes,Yes


In [104]:
Y_test=Y[:3000000]
Y

0             0.000
1             0.000
2             0.000
3             0.000
4             0.000
             ...   
3054343     438.133
3054344     154.553
3054345    2419.729
3054346     121.000
3054347      16.000
Name: sales, Length: 3054348, dtype: float64

In [105]:
test_2=new[3000000:]
test_2

,id,onpromotion,oilprices,const,trend,s27,s37,s47,s57,s67,...,48,49,50,51,52,53,54,No,yes,Yes
3000000,2946540,34,67.714366,1.0,1654.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3000001,2946541,0,67.714366,1.0,1654.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3000002,2946542,73,67.714366,1.0,1654.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3000003,2946543,0,67.714366,1.0,1654.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3000004,2946544,16,67.714366,1.0,1654.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,0,47.570000,1.0,1684.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3054344,3000884,1,47.570000,1.0,1684.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3054345,3000885,148,47.570000,1.0,1684.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3054346,3000886,8,47.570000,1.0,1684.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0


In [106]:
Y_check=Y[3000000:]
Y_check

3000000    2922.000
3000001       0.000
3000002     832.000
3000003       9.000
3000004     864.000
             ...   
3054343     438.133
3054344     154.553
3054345    2419.729
3054346     121.000
3054347      16.000
Name: sales, Length: 54348, dtype: float64

In [107]:
new_col=list(new.columns)
test_col=list(test_1.columns)
#d=test_col-new_col
if new_col==test_col:
    print(True)
print(7)    

7


In [108]:
test_1

,id,onpromotion,oilprices,PETSUPPLIES,CLEANING,HOMEAPPLIANCES,DELI,POULTRY,GROCERYII,MAGAZINES,...,cos1freqM,sin2freqM,cos2freqM,sin3freqM,cos3freqM,sin4freqM,cos4freqM,No,yes,Yes


## Approach
### We can clearly see all the features are independent and also are important for the prediction. so, we'll consider them all and in case of overfitting we'll handle that with the tree parameters itself. we'll be using tweedie or gamma as odjective function as msle does'nt seem good for the data(bcoz of it not being bounded and not convex) and we'll use rmsle as accuracy metric to avoid penalizing when there's significantly higher differences in the observed values than actual values.

In [109]:
xgb.config_context(verbosity=2)

In [110]:
model=xgb.XGBRegressor(objective='reg:tweedie',n_estimators=500,max_depth=20)
model.fit(new_mm,Y)
predictions= model.predict(test_mm)

In [111]:
#Using lightgbm
light=lgb.LGBMRegressor(n_estimators=500,max_depth=8, objective='tweedie',num_leaves=175)#,early_stopping_round=150)
light.fit(new_mm,Y)
predictions= light.predict(test_mm)
print(9)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.133907 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1918
[LightGBM] [Info] Number of data points in the train set: 3054348, number of used features: 108
[LightGBM] [Info] Start training from score 5.883381
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

### we tried both xgboost and lightgbm and we got better results in xgboost. so,we're sticking to that.

## Hyperparameter tuning

In [ ]:
parameters={'max_depth':[6,8,20], 'n_estimators':[75,100,500],'subsample':[0.1,0.5,1],'lambda':[0,0.5,1],'alpha':[0,0.5]}
clf=GridSearchCV(estimator=model, param_grid=parameters,refit=True,scoring='neg_mean_squared_log_error',cv=10)
clf.fit(new,Y)


In [ ]:
predictions=clf.predict(test_1)

In [345]:
metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [ ]:
print(clf.best_params_)
print(clf.best_score_)

In [ ]:
predictions

In [102]:
for i in range(0,len(predictions)):
               if predictions[i]<0:
                  predictions[i]=0
               

In [135]:
for i in range(0,len(predictions)):
               if predictions[i]<0:
                  print(predictions[i])

In [ ]:
msle=mean_squared_log_error(predictions,Y_check)
msle

In [291]:
count = 0
for number in Y_test:
    if number < 0:
        count += 1
count

0

In [292]:
msle

0.2858387109755499

In [293]:
math.sqrt(msle)

0.5346388603305505

In [601]:
count=0
ind=[]
for ele in predictions:
    if ele<0:
        ind.append(count)  
    count+=1
print(ind)

[]


In [595]:
for ele in ind:
    print(test_2.iloc[[ele]])

              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3000001  2946541            0   67.714366    1.0  1654.0     1.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3000001     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3000031  2946571            0   67.714366    1.0  1654.0     1.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3000031     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3000032  2946572            0   67.714366    1.0  1654.0     1.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3000032     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 row

              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3000727  2947267            0   67.714366    1.0  1654.0     1.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3000727     0.0     0.0     0.0  ...   0   0   0   0   0   0   1    0    1   0  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3000749  2947289            0   67.714366    1.0  1654.0     1.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3000749     0.0     0.0     0.0  ...   0   0   0   0   0   0   1    0    1   0  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3000757  2947297            0   67.714366    1.0  1654.0     1.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3000757     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 row

              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3007885  2954425            0       46.73    1.0  1658.0     0.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3007885     0.0     0.0     1.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3007888  2954428            0       46.73    1.0  1658.0     0.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3007888     0.0     0.0     1.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3007918  2954458            0       46.73    1.0  1658.0     0.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3007918     0.0     0.0     1.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 row

              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3008647  2955187            0       45.78    1.0  1659.0     0.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3008647     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3008677  2955217            0       45.78    1.0  1659.0     0.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3008677     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3008680  2955220            0       45.78    1.0  1659.0     0.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3008680     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 row

              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3009271  2955811            0       45.78    1.0  1659.0     0.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3009271     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3009274  2955814            0       45.78    1.0  1659.0     0.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3009274     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3009304  2955844            0       45.78    1.0  1659.0     0.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3009304     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 row

...

              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3037126  2983666            0   67.714366    1.0  1675.0     1.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3037126     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3037156  2983696            0   67.714366    1.0  1675.0     1.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3037156     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3037159  2983699            0   67.714366    1.0  1675.0     1.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3037159     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 row

              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3038479  2985019            0       49.37    1.0  1676.0     0.0     1.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3038479     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3038512  2985052            0       49.37    1.0  1676.0     0.0     1.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3038512     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3038542  2985082            0       49.37    1.0  1676.0     0.0     1.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3038542     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 row

              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3046626  2993166            0       48.81    1.0  1680.0     0.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3046626     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    0   1  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3046627  2993167            0       48.81    1.0  1680.0     0.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3046627     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    0   1  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3046630  2993170            0       48.81    1.0  1680.0     0.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3046630     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    0   1  

[1 row

              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3049762  2996302            0   67.714366    1.0  1682.0     1.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3049762     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3049765  2996305            0   67.714366    1.0  1682.0     1.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3049765     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3049795  2996335            0   67.714366    1.0  1682.0     1.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3049795     0.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    1   0  

[1 row

              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3053491  3000031            0       47.57    1.0  1684.0     0.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3053491     1.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    0   1  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3053494  3000034            0       47.57    1.0  1684.0     0.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3053494     1.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    0   1  

[1 rows x 109 columns]
              id  onpromotion  oil prices  const   trend  s(2,7)  s(3,7)  \
3053510  3000050            0       47.57    1.0  1684.0     0.0     0.0   

         s(4,7)  s(5,7)  s(6,7)  ...  48  49  50  51  52  53  54  Yes  yes  No  
3053510     1.0     0.0     0.0  ...   0   0   0   0   0   0   0    0    0   1  

[1 row

In [250]:
ans=pd.DataFrame(predictions)
submit=pd.DataFrame(columns=['id'])
submit['id']=test['id']
submit['sales']=ans[0]
submit

,id,sales
0,3000888,5.484168
1,3000889,0.019731
2,3000890,5.574053
3,3000891,2637.834717
4,3000892,0.119606
...,...,...
28507,3029395,372.060822
28508,3029396,89.641861
28509,3029397,1155.353516
28510,3029398,77.639328


In [252]:
submit.to_csv(r"C:\Users\Dell\Documents\projects\sales1\submit14.csv",index=False)

### after submiting the predictions we got rmsle of 0.43489 which was at the rank of 81 out of around 700 entries. the highest accuracy one got was 0.3762 which is not so different from ours considering the metric we used. to improve the model we should consider using some better ensemble techniques. 